In [131]:
import os
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score
from scipy.stats import norm
import scipy.optimize as opt
import yfinance as yf
import nasdaqdatalink
import quantstats as qs
from scipy import interpolate
from scipy import ndimage
import datetime as dt
import bt
import trading_calendars as tc
import pandas_market_calendars as mcal

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, iplot

# Buy and Hold

## Portfolio 1 40% fixed income (20% SHY, 20% TLT) 40% equities (20% VTI, 20% IWN) and 20% gold (GLD)

In [4]:
tickers = yf.Tickers('shy tlt vti iwn gld')
df = tickers.history(period='max')
df = df.dropna(how='any', axis=0, inplace=False)

[*********************100%***********************]  5 of 5 completed

3 Failed downloads:
- ^VTI: No data found for this date range, symbol may be delisted
- ^IWN: No data found for this date range, symbol may be delisted
- ^GLD: No data found for this date range, symbol may be delisted


In [44]:
close_df = df['Close'].copy(deep=True)

target_df = close_df.copy(deep=True)
target_df = target_df*0 + 0.2

s = bt.Strategy('Portfolio1', [bt.algos.SelectAll(),
                           bt.algos.WeighTarget(target_df),
                           bt.algos.Rebalance()])

t = bt.Backtest(s, close_df)
res= bt.run(t)

In [45]:
res.display()

Stat                 Portfolio1
-------------------  ------------
Start                2004-11-17
End                  2022-03-11
Risk-free rate       0.00%

Total Return         272.87%
Daily Sharpe         0.89
Daily Sortino        1.43
CAGR                 7.90%
Max Drawdown         -20.16%
Calmar Ratio         0.39

MTD                  -1.01%
3m                   -2.91%
6m                   -1.39%
YTD                  -3.95%
1Y                   2.60%
3Y (ann.)            11.16%
5Y (ann.)            8.66%
10Y (ann.)           6.77%
Since Incep. (ann.)  7.90%

Daily Sharpe         0.89
Daily Sortino        1.43
Daily Mean (ann.)    8.02%
Daily Vol (ann.)     9.00%
Daily Skew           -0.35
Daily Kurt           9.07
Best Day             4.21%
Worst Day            -4.89%

Monthly Sharpe       1.04
Monthly Sortino      1.88
Monthly Mean (ann.)  7.88%
Monthly Vol (ann.)   7.54%
Monthly Skew         -0.65
Monthly Kurt         3.51
Best Month           7.12%
Worst Month          -10.59%

In [50]:
res.plot()

<AxesSubplot:title={'center':'Equity Progression'}>

## Portfolio 2 70% in bonds (AGG), 25% in cash (BIL), and 5% in REITs (VNQ)

In [51]:
tickers = yf.Tickers('agg bil vnq')
df = tickers.history(period='max')
df = df.dropna(how='any', axis=0, inplace=False)

[*********************100%***********************]  3 of 3 completed


In [52]:
close_df = df['Close'].copy(deep=True)

target_df = close_df.copy(deep=True)
target_df = target_df*0 + 0.2

s = bt.Strategy('Portfolio1', [bt.algos.SelectAll(),
                           bt.algos.WeighTarget(target_df),
                           bt.algos.Rebalance()])

t = bt.Backtest(s, close_df)
res= bt.run(t)

In [53]:
res.display()

Stat                 Portfolio1
-------------------  ------------
Start                2007-05-29
End                  2022-03-11
Risk-free rate       0.00%

Total Return         53.31%
Daily Sharpe         0.48
Daily Sortino        0.76
CAGR                 2.93%
Max Drawdown         -16.60%
Calmar Ratio         0.18

MTD                  -0.10%
3m                   -2.01%
6m                   -1.25%
YTD                  -3.10%
1Y                   2.86%
3Y (ann.)            3.30%
5Y (ann.)            2.96%
10Y (ann.)           2.71%
Since Incep. (ann.)  2.93%

Daily Sharpe         0.48
Daily Sortino        0.76
Daily Mean (ann.)    3.10%
Daily Vol (ann.)     6.43%
Daily Skew           0.03
Daily Kurt           14.93
Best Day             3.40%
Worst Day            -3.57%

Monthly Sharpe       0.61
Monthly Sortino      1.01
Monthly Mean (ann.)  3.02%
Monthly Vol (ann.)   4.98%
Monthly Skew         -0.38
Monthly Kurt         6.07
Best Month           6.31%
Worst Month          -6.94%

Y

# Tactical Allocation

## Portfolio 3: 20% equal weight in VTI VEU VNQ AGG DBC

Compute the 10-day MA at the end of each month. If price > 10-day allocate the 20% in the given investment else cash.

In [77]:
tickers = yf.Tickers('vti veu vnq agg dbc bil')
df = tickers.history(period='max')
df = df.dropna(how='any', axis=0, inplace=False)

[*********************100%***********************]  6 of 6 completed


In [78]:
close_df = df['Close'].copy(deep=True)
ma_df = df['Close'].rolling(10).mean().dropna()

In [198]:
dateIndex = close_df.reset_index()['Date']

dateIndex = pd.DataFrame(dateIndex)
dateIndex['YEAR'] = dateIndex['Date'].apply(lambda x: x.year)
dateIndex['MONTH'] = dateIndex['Date'].apply(lambda x: x.month)
dateIndex['DAY'] = dateIndex['Date'].apply(lambda x: x.day)
dateIndex = dateIndex.groupby(['YEAR', 'MONTH']).max()['Date'].values

target_df = pd.DataFrame(index=dateIndex, columns=close_df.columns)
target_df = target_df.iloc[1:,:]

In [201]:
for i in target_df.index:
    bil_alloc = 0.0
    for tick in ['VTI', 'VEU', 'VNQ', 'AGG', 'DBC']:
        if ma_df.loc[i, tick] > close_df.loc[i, tick]:
            target_df.loc[i, tick] = 0.2
        else:
            bil_alloc += 0.2
            target_df.loc[i, tick] = 0.0
            
    target_df.loc[i, "BIL"] = bil_alloc

In [207]:
s = bt.Strategy('Portfolio3', [bt.algos.SelectAll(),
                           bt.algos.WeighTarget(target_df),
                           bt.algos.Rebalance()])

t = bt.Backtest(s, close_df)
res= bt.run(t)

In [208]:
res.display()

Stat                 Portfolio3
-------------------  ------------
Start                2007-05-29
End                  2022-03-11
Risk-free rate       0.00%

Total Return         60.62%
Daily Sharpe         0.36
Daily Sortino        0.53
CAGR                 3.26%
Max Drawdown         -32.60%
Calmar Ratio         0.10

MTD                  -1.30%
3m                   0.02%
6m                   3.36%
YTD                  -1.93%
1Y                   5.42%
3Y (ann.)            3.94%
5Y (ann.)            3.31%
10Y (ann.)           3.65%
Since Incep. (ann.)  3.26%

Daily Sharpe         0.36
Daily Sortino        0.53
Daily Mean (ann.)    3.74%
Daily Vol (ann.)     10.27%
Daily Skew           -0.87
Daily Kurt           34.54
Best Day             6.55%
Worst Day            -9.11%

Monthly Sharpe       0.40
Monthly Sortino      0.59
Monthly Mean (ann.)  3.61%
Monthly Vol (ann.)   9.05%
Monthly Skew         -1.00
Monthly Kurt         10.77
Best Month           10.97%
Worst Month          -13.57%